In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request

In [2]:
base_url = "https://veritasihomes.com/premium-homes/"
extend = ["camberwall-advantage/", "camberwall-court",]

In [3]:
#get url, find name, location, description and append them to their respective list
names = []
locations = []
descriptions = []
dfs = []
images = []


for i in extend:
    url = base_url+i
    req = Request(url, headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac \
                                  OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'})
    resp = urlopen(req)
    html = resp.read()
    soup = bs(html, "lxml")
    
    df = pd.read_html(html)
    for d in df:
        dfs.append(d.to_dict())
        
    name = soup.find("h2", class_= "elementor-heading-title").string
    names.append(name)
        
    location = soup.find("span", class_="elementor-icon-list-text").string
    locations.append(location)
        
        
    description = soup.find("div", class_ = "elementor-text-editor" ).text.strip()
    descriptions.append(description)
        
    image = soup.find_all("img", class_ = "swiper-slide-image")
    for i in image:
        images.append(i.get("src"))
            


In [4]:
#get the virtual inspection links
url_i = "https://veritasihomes.com/virtualinspection/"
req1 = Request(url_i, headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac \
OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'})
resp1 = urlopen(req1)
html1 = resp1.read()
soup1 = bs(html1, "lxml")

virtual_tour = []
for link in soup1.find_all("a" , class_="elementor-flip-box__layer elementor-flip-box__back"):
    virtual_tour.append(link.get("href"))

In [5]:
data1 = {"name": names[0],
              "location":locations[0],
              "description": descriptions[0],
             "payment plans": {
                 "advantage 1":{
                 "2 bedroom payment plan": dfs[0],
                 "3 bedroom payment plan": dfs[1]
             },
         "advantage 2":{
             "2 bedroom payment plan": dfs[2],
             "3 bedroom payment plan": dfs[3],
             "4 bedroom payment plan": dfs[4]
         }},
         "images":images[:5]
        }

In [6]:
data2 = {"name": names[1],
              "location":locations[1],
              "description": descriptions[1],
              "payment plans": {
                  "2 bedroom payment plan": dfs[5],
                  "3 bedroom payment plan": dfs[6],
                  "4 bedroom semi-detached payment plan": dfs[7],
                  "3 bedroom terraces payment plan": dfs[8],
                  "3 bedroom bungalow payment plan": dfs[9]
              },
         "images":images[5:]
                                   }

In [7]:
all_data = {"premium plan":{"home 1": data1,
                           "home 2": data2},
            "virtual tours":{
                "camberwall apartments": virtual_tour[0],
                "camberwall terraces": virtual_tour[1],
                "camberwall bungalow": virtual_tour[2]
            }
           }

In [8]:
print(all_data)

{'premium plan': {'home 1': {'name': 'Camberwall Advantage', 'location': 'IKATE LEKKI, LAGOS.', 'description': 'Camberwall Advantage is an upscale real estate development project by Veritasi Homes and Properties. Comprising apartments and units of flat, Camberwall Advantage is strategically located at Ikate.\xa0The proposed project features unique and elegant designs tailored to meet the need of the modern day investor as well as appeal to aesthetics. With the existing reputation of Veritasi Homes with Camberwall projects, this project promises homes of a high standard, quality finishings and best value for investment.\xa0Our strategy transcends developing homes to solve the housing problems of Nigeria, we have injected interesting metrics to help our clients get cash flow into our product plan. With this,\xa0 Camberwall Advantage is set to set a new hallmark for a sophisticated urban living.Ikate is one of the most developed neighbourhoods in Lagos Island, just about five minutes driv

In [9]:
import json

with open('veritasi.json', 'w') as f:
    json.dump(all_data, f, indent = 2)